
* *** 원본 소스는 홍콩과기대 김성훈 교수님의  [Github link](https://github.com/hunkim/DeepLearningZeroToAll)를 참조하세요 ***     
* *** Watson studio notebook 에서 작업 가능하도록 수정하고 제가 study 하면서 이해한 내용을 첨언하였습니다.***


## Lab-02-2 linear regression feed

* ***바로 앞에 Lab 2-1 하고 처리하는 로직은 동일합니다 ***
* ***단, placeholder 라는 함수를 사용해서  input value 를 변수처리를 하도록 했고 ***
* ***앞에서 얘기했듯이 이제 예측을 한번 해보는 겁니다.***

In [2]:
import tensorflow as tf
tf.set_random_seed(777)  # for reprducibilty

C:\Users\HyeonSuPARK\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


* ***W,b 에 이름을 부여하고, 시작 값을 random으로 생성합니다.***

In [3]:
# Try to find value for W and b to compute y_data = x_data * W + b  
# We know that W should be 1 and b should be 0
# But let's TensorFlow figure it out 
W = tf.Variable(tf.random_normal([1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

* ***값이 어떻게 들어올지 모르니, 행렬의 크기를 None, 즉 모른다로 정의합니다.***

In [4]:
#### Now we can use X and Y in place of x_data and y_data
#### placeholders for a tensor that will be always fed using feed_dict
#### See http://stackoverflow.com/questions/36693740/
X = tf.placeholder(tf.float32, shape=[None])
Y = tf.placeholder(tf.float32, shape=[None])

* ***앞에 Lab2-1과 동일합니다.***

In [5]:
# Out hypothesis XW+b
hypothesis = X * W + b

# cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

In [6]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)

In [7]:
# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

* ***Lab2-1과 동일하지만, run 할때 training data를 받습니다.***

In [8]:
for step in range(2001):
    cost_val, W_val, b_val, _ =  sess.run([cost, W, b, train], feed_dict={X: [1, 2, 3], Y: [1, 2, 3]})
    if step % 200 == 0:
        print(step, cost_val, W_val, b_val)


0 3.5240757 [2.1286771] [-0.8523567]
200 0.07030439 [1.3072149] [-0.6983712]
400 0.026845641 [1.18984] [-0.4315508]
600 0.010250964 [1.1173096] [-0.26667204]
800 0.003914324 [1.0724902] [-0.16478711]
1000 0.0014946823 [1.0447946] [-0.10182849]
1200 0.0005707396 [1.0276802] [-0.06292368]
1400 0.00021793543 [1.0171047] [-0.03888312]
1600 8.322027e-05 [1.0105698] [-0.02402747]
1800 3.1776715e-05 [1.0065314] [-0.01484741]
2000 1.21343355e-05 [1.0040361] [-0.00917497]


##### 결과는 1,0 으로 수렴하지요. 

###  자 ! 이제 우리의 모델을 가지고 시험(예측)을 해보겠습니다.

In [9]:
print(sess.run(hypothesis, feed_dict={X: [5]}))
print(sess.run(hypothesis, feed_dict={X: [2.5]}))
print(sess.run(hypothesis, feed_dict={X: [1.5, 3.5]}))

[5.0110054]
[2.500915]
[1.4968792 3.5049512]


* ***주어진 x 값과 동일한 y 값이 나와야지요***
* ***여기서 오해하면 안되는 것이,,, W,b 가 근사치로 정해졌기 때문에  예측치도 정확하게 일치 하지는 않습니다.***

### 자 ! 이제 새로운 training 데이터를 가지고, 손쉽게  새로운 모델을 만들어 보겠습니다.

In [11]:
for step in range(2001):
    cost_val, W_val, b_val, _ = \
        sess.run([cost, W, b, train],
                 feed_dict={X: [1, 2, 3, 4, 5],
                            Y: [2.1, 3.1, 4.1, 5.1, 6.1]})
    if step % 200 == 0:
        print(step, cost_val, W_val, b_val)

0 2.5203167e-07 [1.000325] [1.0988272]
200 6.5212554e-08 [1.0001651] [1.0994035]
400 1.6849981e-08 [1.000084] [1.0996966]
600 4.380945e-09 [1.0000429] [1.0998453]
800 1.1582415e-09 [1.0000219] [1.0999204]
1000 3.2878233e-10 [1.0000118] [1.0999575]
1200 7.504468e-11 [1.0000058] [1.0999795]
1400 7.504468e-11 [1.0000058] [1.0999795]
1600 7.504468e-11 [1.0000058] [1.0999795]
1800 7.504468e-11 [1.0000058] [1.0999795]
2000 7.504468e-11 [1.0000058] [1.0999795]


* ***새로운 W,b가 구해졌나요 ? ***
* ***다시 테스트 (예측) 해봅시다.***

In [12]:
print(sess.run(hypothesis, feed_dict={X: [5]}))
print(sess.run(hypothesis, feed_dict={X: [2.5]}))
print(sess.run(hypothesis, feed_dict={X: [1.5, 3.5]}))

[6.1000085]
[3.5999942]
[2.5999885 4.6      ]


### 지금 두개의 모델을 만들었습니다.  만약 이 모델을  저장, 관리하면서 다른 사용자들에게 공유하고 싶다면 ?
** [IBM CLoud site](https://console.bluemix.net/catalog/services/machine-learning) 에서 Watson machine learning 서비스를 활용하시면 됩니다. 